In [1]:
import numpy as np
from src.rowwise_metrics import mutual_information, euclid, cosine
from src.simulation import get_spectrum

In [2]:
from SimulatedLIBS.simulation import SimulatedLIBS

"""
spectrum = get_spectrum(
    Te=1.0,
    Ne=10**17,
    elements=['Fe'],
    percentages=[100],
    resolution=10000,
    low_w=200,
    upper_w=1000,
    max_ion_charge=3,
    webscraping='dynamic',
    return_wavelength=True,
)
"""

"\nspectrum = get_spectrum(\n    Te=1.0,\n    Ne=10**17,\n    elements=['Fe'],\n    percentages=[100],\n    resolution=10000,\n    low_w=200,\n    upper_w=1000,\n    max_ion_charge=3,\n    webscraping='dynamic',\n    return_wavelength=True,\n)\n"

In [22]:
from SimulatedLIBS.simulation import SimulatedLIBS

PARAMS = dict(
    Te=1.0,
    Ne=10**18,
    resolution=10000,
    low_w=200,
    upper_w=1000,
    max_ion_charge=3,
    webscraping='dynamic',
)

n = 20
elements = sorted(['Fe', 'Co']) # WARNING! sorts the elements in alphabetical order to prevent duplicities
path = f'data/{elements}_{n}.npy'
try:
    spectra = np.load(open(path, 'rb'), allow_pickle=True) # TODO allow pickle ?
except OSError:
    if not input("Data not found. Input empty string to dowload the data."):
        spectra = np.array(list(map(
            lambda x: get_spectrum(
                elements=elements,
                percentages=[x,100-x],
                **PARAMS,
            ),
            np.linspace(0, 100, n), 
        )))
        np.save(open(path, 'wb'), spectra)

['Co', 'Fe']


In [23]:
del spectra

In [ ]:
spectra = spectra[::-1, :]
np.save(open(f'data/{elements}_{n}.npy', 'wb'), spectra)

In [5]:
metrics = [euclid, cosine, mutual_information]
matrices = [np.zeros((n, n)) for _ in metrics]
for i in range(len(metrics)):
    for j in range(n):
        for k in range(n):
            matrices[i][j][k] = metrics[i](spectra[j], spectra[k])

ValueError: operands could not be broadcast together with shapes (22125,) (109393,) 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

a = np.arange(10).reshape((1, -1))
b = (np.arange(10).reshape((1, -1)) + 1) * 3
print(cosine_similarity(a, b)[0][0] + cosine(a[0], b[0]))

1.0


In [ ]:
import plotly.express as px
px.imshow(matrices[0])

In [ ]:
plot_spectra([spectra[0], spectra[-2]])

In [ ]:
from libs_tools.visualization.spectral import plot_spectra

matrices[0] = matrices[0] / matrices[0][0].max()
matrices[2] = matrices[2] - matrices[2][0].min()
plot_spectra([matrix[0] for matrix in matrices], labels=[foo.__name__ for foo in metrics])